In [3]:
%pip install matplotlib regex wordcloud pillow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import regex
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# ---------------------------
# FUNCTION: check date pattern
# ---------------------------
def startsWithDateAndTime(s):
    pattern = r"^\d{1,2}\/\d{1,2}\/\d{2,4},\s\d{1,2}:\d{2}"
    return bool(regex.match(pattern, s))

# ---------------------------
# FUNCTION: break full line into (date, time, author, message)
# ---------------------------
def parseLine(line):
    date_time, msg = line.split(" - ", 1)
    date, time = date_time.split(", ")
    
    if ": " in msg:
        author, message = msg.split(": ", 1)
    else:
        author, message = "System Message", msg
    return date, time, author, message

# ---------------------------
# MAIN ANALYSIS FUNCTION
# ---------------------------
def analyze_chat(filepath, output_box):

    parsed = []
    buffer = []
    date = time = author = None

    with open(filepath, "r", encoding="utf-8") as fp:
        fp.readline()

        while True:
            line = fp.readline()
            if not line:
                break

            line = line.strip()

            # New message starts here
            if startsWithDateAndTime(line):
                if buffer:
                    parsed.append([date, time, author, " ".join(buffer)])
                buffer.clear()

                date, time, author, msg = parseLine(line)
                buffer.append(msg)

            else:
                buffer.append(line)

    # Add last message
    if buffer:
        parsed.append([date, time, author, " ".join(buffer)])

    # Count authors
    authors = [row[2] for row in parsed]
    count_authors = Counter(authors)

    # Count total messages
    total_messages = len(parsed)

    # Count words
    all_msgs = " ".join([row[3] for row in parsed])
    total_words = len(all_msgs.split())

    # Show stats
    output_box.delete(1.0, tk.END)
    output_box.insert(tk.END, "Chat Loaded Successfully!\n\n")
    output_box.insert(tk.END, f"Total Messages: {total_messages}\n")
    output_box.insert(tk.END, f"Total Words: {total_words}\n\n")
    output_box.insert(tk.END, "Messages Per Person:\n")

    for a, c in count_authors.items():
        output_box.insert(tk.END, f"  {a}: {c}\n")

    # Wordcloud
    wc = WordCloud(width=800, height=400, background_color="white").generate(all_msgs)
    plt.figure(figsize=(10,5))
    plt.imshow(wc)
    plt.axis("off")
    plt.show()


# -----------------------------------------
# GUI
# -----------------------------------------

root = tk.Tk()
root.title("WhatsApp Chat Analyzer")
root.geometry("800x600")
root.config(bg="#1e1e1e")

title = tk.Label(
    root,
    text="WhatsApp Chat Analyzer",
    font=("Arial", 22, "bold"),
    bg="#1e1e1e",
    fg="white"
)
title.pack(pady=10)

output_box = scrolledtext.ScrolledText(
    root, width=90, height=25, font=("Arial", 10), bg="#2c2c2c", fg="white"
)
output_box.pack(pady=10)

# ---------------------------
# BUTTON: Select File
# ---------------------------
def pick_file():
    filepath = filedialog.askopenfilename(
        title="Select WhatsApp Chat Text File",
        filetypes=[("Text Files", "*.txt")]
    )

    if filepath:
        output_box.insert(tk.END, f"\nSelected File: {filepath}\n")
        analyze_chat(filepath, output_box)
    else:
        messagebox.showwarning("Warning", "No file selected!")

btn = tk.Button(
    root,
    text="📂 Select WhatsApp Chat File",
    font=("Arial", 14, "bold"),
    bg="#4CAF50",
    fg="white",
    padx=20,
    pady=10,
    command=pick_file
)
btn.pack()

root.mainloop()


C:\Users\SHIVAM SHARMA\AppData\Local\Temp\ipykernel_13556\3003997515.py:89: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
